In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [3]:
# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#from scipy.stats import shapiro, kstest, poisson, chisquare, ttest_ind, levene, bartlett, sem, ppf
import scipy.stats as stats
from scipy.stats import shapiro, levene
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

In [14]:
def exploracion(df):
    """
    Realiza una exploración inicial y descriptiva de un DataFrame para identificar información relevante
    sobre los datos, como valores nulos, duplicados, tipos de datos, y estadísticas generales.
    La función también imprime un resumen de las columnas categóricas y numéricas del DataFrame.
    Parámetros:
    -----------
    dataframe (DataFrame): El DataFrame que contiene los datos.
    Descripción:
    ------------
    La función realiza los siguientes análisis y genera la siguiente información:
    - El número de filas y columnas del DataFrame.
    - La cantidad de datos duplicados y su porcentaje en relación al total.
    - La cantidad de columnas con valores nulos y las columnas sin valores nulos.
    - El tipo de dato de cada columna.
    - El número de valores únicos por columna.
    Además, la función imprime estadísticas descriptivas de las columnas categóricas (si las hay) y
    numéricas (si las hay) del DataFrame."""
    df_info = pd.DataFrame()
    df_info["% nulos"] = round(df.isna().sum() / df.shape[0] * 100, 2).astype(str) + "%"
    df_info["% no_nulos"] = (
        round(df.notna().sum() / df.shape[0] * 100, 2).astype(str) + "%"
    )
    df_info["tipo_dato"] = df.dtypes
    df_info["num_valores_unicos"] = (
        df.nunique()
    )  # devuelve el número de valores únicos en la columna.
    print(f"""El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.
Tiene {df.duplicated().sum()} datos duplicados, lo que supone un porcentaje de {round(df.duplicated().sum()/df.shape[0]*100, 2)}% de los datos.
Hay {len(list(df_info[(df_info["% nulos"] != "0.0%")].index))} columnas con datos nulos, y son:
{list(df_info[(df_info["% nulos"] != "0.0%")].index)}
y sin nulos hay {len(list(df_info[(df_info["% nulos"] == "0.0%")].index))} columnas y son:
{list(df_info[(df_info["% nulos"] == "0.0%")].index)}
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:""")
    print("Principales estadísticos de las columnas categóricas:")
    if (
        df.select_dtypes(include="object").shape[1] > 0
    ):  # Verificar si hay columnas categóricas
        display(df.describe(include="O").T)
    else:
        print("No hay columnas categóricas en el DataFrame.")
    print("Principales estadísticos de las columnas numéricas:")
    if (
        df.select_dtypes(exclude="object").shape[1] > 0
    ):  # Verificar si hay columnas numéricas
        display(df.describe(exclude="O").T)
    else:
        print("No hay columnas numéricas en el DataFrame.")
    return df_info

In [ ]:
## https://www.kaggle.com/datasets/dante890b/wikipedia-serial-killers-list?select=serial_killers_by_victims.csv : 

In [8]:
df_victimas = pd.read_csv('serial_killers_by_victims.csv')
df_victimas.head(3)

,Name,Country,Start year,End year,Proven victims,Possible victims,Notes
0,Luis Garavito,"Colombia, Ecuador, Venezuela",1992.0,1999.0,193.0,247.0,"Child-murderer, torture-killer, and rapist kno..."
1,Pedro López,"Colombia, Peru, Ecuador",1969.0,1980.0,110.0,300.0,"Child-murderer and rapist, known as ""The Monst..."
2,Javed Iqbal,Pakistan,1998.0,1999.0,100.0,100.0,"Child-murderer and rapist, known as ""Kukri"", I..."


In [11]:
df_victimas.columns

Index(['Name', 'Country', 'Start year', 'End year', 'Proven victims',
       'Possible victims', 'Notes'],
      dtype='object')

In [13]:
exploracion(df_victimas)

El DataFrame tiene 757 filas y 7 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.

Hay 2 columnas con datos nulos, y son: 
['Start year', 'Notes']

y sin nulos hay 5 columnas y son: 
['Name', 'Country', 'End year', 'Proven victims', 'Possible victims']


A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Name,0.0%,100.0%,object,757
Country,0.0%,100.0%,object,113
Start year,2.51%,97.49%,float64,120
End year,0.0%,100.0%,float64,115
Proven victims,0.0%,100.0%,float64,67


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
Name,757,757,Luis Garavito,1
Country,757,113,United States,275
Notes,754,754,"Child-murderer, torture-killer, and rapist kno...",1


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Start year,738.0,1978.682927,25.774773,1871.0,1970.0,1984.0,1996.0,2023.0
End year,757.0,1941.579921,293.529192,9.0,1976.0,1992.0,2003.0,2032.0
Proven victims,757.0,13.675033,36.668291,1.0,4.0,7.0,13.0,810.0
Possible victims,757.0,24.762219,57.756596,2.0,5.0,11.0,20.0,1000.0


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Name,0.0%,100.0%,object,757
Country,0.0%,100.0%,object,113
Start year,2.51%,97.49%,float64,120
End year,0.0%,100.0%,float64,115
Proven victims,0.0%,100.0%,float64,67
Possible victims,0.0%,100.0%,float64,91
Notes,0.4%,99.6%,object,754


In [39]:
df_victimas['Proven victims'].unique()

array([193., 110., 100.,  83.,  72.,  71.,  70.,  67.,  65.,  60.,  57.,
        54.,  53.,  52.,  50.,  49.,  45.,  42.,  41.,  38.,  37.,  36.,
        35.,  33.,  32.,  31.,  30.,  28.,  27.,  26.,  25.,  24.,  23.,
        22.,  21.,  20.,  19.,  18.,  17.,  16.,  15.,  14.,  13.,  12.,
        11.,  10.,   9.,   8.,   7.,   6.,   5.,   4.,   3.,   2.,   1.,
       218.,  85.,  48.,  43.,  29., 400., 114.,  91.,  77.,  62.,  40.,
       810.])

In [29]:
df_victimas['Possible victims'].unique()

array([ 247.,  300.,  100.,   86.,  180.,   70.,   67.,   93.,   57.,
         54.,   56.,   52.,   50.,   80.,   60.,   42.,   41.,   76.,
         49.,   36.,   35.,  137.,   39.,   33.,   32.,   31.,   38.,
         30.,   45.,   28.,  177.,   26.,   25.,   27.,   24.,  150.,
         23.,   88.,   22.,   43.,   21.,   20.,   19.,   44.,   40.,
         18.,   92.,   17.,   16.,   15.,   29.,   14.,   46.,   13.,
         12.,   11.,   10.,   79.,   34.,    9.,  160.,  130.,  112.,
          8.,   48.,    7.,    6.,   37.,    5.,   82.,    4.,  175.,
          3.,   90.,    2.,  250.,  200.,   72.,  400.,   63.,  259.,
         84.,   47.,  131., 1000.,  114.,   91.,   62.,   73.,  124.,
        620.])

In [ ]:
## https://www.kaggle.com/datasets/vesuvius13/serial-killers-dataset?select=5_to_14_victim_count.csv: 

In [43]:
df_1 = pd.read_csv('1.csv')

In [44]:
df_2 = pd.read_csv('2.csv')

In [45]:
df_3 = pd.read_csv('3.csv')

In [46]:
df_4 = pd.read_csv('4.csv')

In [53]:
df_unido = pd.concat([df_1, df_2,df_3,df_4],axis=0)
df_unido

,Name,Country,Years active,Proven victims,Possible victims,Notes
0,Carl Eugene Watts,United States,1974 to 1982,14,80+,Believed to have killed over 80 women in multi...
1,Philipp Tyurin,Soviet Union,1945 to 1946,14,29,"Known as the ""Leningrad Maniac"" and the ""Hellr..."
2,Zdzisław Marchwicki,Poland,1964 to 1970,14,21+,Also known as Vampire of Zagłębie. Killed 14 w...
3,Monster of Florence,Italy,1968 to 1985,14,16,Unidentified killer who shot couples in lovers...
4,Joachim Kroll,West Germany,1955 to 1976,14,NaN,"Known as the ""Ruhr Cannibal"" and ""The Duisburg..."
...,...,...,...,...,...,...
24,Leonarda Cianciulli,Italy,1939 to 1940,3,-,"Murdered three women in the town of Correggio,..."
25,Özkan Zengin,Turkey,2008,3,4,Slit the throats of three confirmed victims be...
26,Albert Fish,United States,1924 to 1932,3,9-100+,American serial killer and child rapist who co...
27,Heriberto Seda,United States,1990 to 1996,3,8+,Considered to be a copycat of the Zodiac Kille...


In [54]:
df_unido.shape

(305, 6)

In [ ]:
## Comparamos los dos: 

In [55]:
exploracion(df_victimas)

El DataFrame tiene 757 filas y 7 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 2 columnas con datos nulos, y son:
['Start year', 'Notes']
y sin nulos hay 5 columnas y son:
['Name', 'Country', 'End year', 'Proven victims', 'Possible victims']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:
Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
Name,757,757,Luis Garavito,1
Country,757,113,United States,275
Notes,754,754,"Child-murderer, torture-killer, and rapist kno...",1


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Start year,738.0,1978.682927,25.774773,1871.0,1970.0,1984.0,1996.0,2023.0
End year,757.0,1941.579921,293.529192,9.0,1976.0,1992.0,2003.0,2032.0
Proven victims,757.0,13.675033,36.668291,1.0,4.0,7.0,13.0,810.0
Possible victims,757.0,24.762219,57.756596,2.0,5.0,11.0,20.0,1000.0


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Name,0.0%,100.0%,object,757
Country,0.0%,100.0%,object,113
Start year,2.51%,97.49%,float64,120
End year,0.0%,100.0%,float64,115
Proven victims,0.0%,100.0%,float64,67
Possible victims,0.0%,100.0%,float64,91
Notes,0.4%,99.6%,object,754


In [68]:
df_victimas['Name'].unique()

array(['Luis Garavito', 'Pedro López', 'Javed Iqbal', 'Mikhail Popkov',
       'Daniel Camargo Barbosa', 'Pedro Rodrigues Filho',
       'Kampatimar Shankariya', 'Hoshang Amini', 'Yang Xinhai',
       'Abul Djabar', 'Samuel Little', 'William Unek', 'Mohammed Bijeh',
       'Andrei Chikatilo', 'Anatoly Onoprienko',
       'Florisvaldo de Oliveira', 'Gary Ridgway', 'Alexander Pichushkin',
       'Wang Qiang', 'Ahmad Suradji', 'Raman Raghav', 'Moses Sithole',
       'Serhiy Tkach', 'Gennady Mikhasevich', 'Hadj Mohammed Mesfewi',
       'Vera Renczi', 'Fernando Hernández Leyva', 'John Wayne Gacy',
       'Ali Asghar Borujerdi', 'Vasili Komaroff',
       'Ramadan Abdel Rehim Mansour', 'Radik Tagirov', 'Karl Denke',
       'Francisco das Chagas Rodrigues de Brito',
       'Monster of the Mangones', 'Milton Sipalo',
       'Luis Gregorio Ramírez Maestre', 'David Thabo Simelane',
       'Zhang Jun', 'Mariam Soulakiotis', 'Cedric Maake',
       'Robert Pickton', 'Bruce George Peter Lee', 'The S

In [ ]:
## Género: 



In [70]:
df_unido['Name'].unique()

array(['Carl Eugene Watts', 'Philipp Tyurin', 'Zdzisław Marchwicki',
       'Monster of Florence', 'Joachim Kroll', 'Arthur Shawcross',
       'The Doodler', 'Marcelo Costa de Andrade', 'Julio Pérez Silva',
       'Sergey Shipilov', 'Abdufatto Zamanov', 'Jeong Nam-gyu',
       'Amir Qayyum', 'Jake Bird', 'Belle Gunness',
       'Cleveland Torso Murderer', 'Kaspars Petrovs', 'Richard Ramirez',
       'Sleepy Hollow Killer', 'Peter Sutcliffe',
       'Francisco Antonio Laureana', 'Tamara Samsonova', 'Herbert Mullin',
       'Boston Strangler', 'Lorenzo Gilyard', 'Joseph James DeAngelo',
       'Vladimir Storozhenko', 'Vasiliy Kulik', 'Nikolai Dudin',
       'Denis Pischikov', 'Nikolay Shubin', 'Johannes Mashiane',
       'Mukosi Freddy Mulaudzi', 'Thozamile Taki', 'Rainbow Maniac',
       'Naceur Damergi', 'Adolf Seefeldt', 'Donald Henry Gaskins',
       'William Suff', 'Vladimir Romanov', 'Maury Travis',
       'Dennis Nilsen', 'Kenneth Bianchi', 'Charles Sobhraj',
       'Abdallah al-H